# Generate GAFAM data using daily data service - C#

### Overview
GAFAM is an acronym for five popular U.S. tech stocks: Google (Alphabet Inc), Apple, Facebook, Amazon, and Microsoft.  

This sample demonstrates how to directly request and plot simultaneously `daily data` for GAFAM instruments and index.<br>

It enables to retrieve **on-demand** data by calling a dedicated daily data service. 

### Inputs/outputs
GAFAM sample requires instruments' identifier as per input. It:
* retrives close prices from open-high-low-close (OHLC) and volume data
* computes GAFAM index

This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve daily data from the dedicated hosted service. The queried endpoint in this script are:
* *DailyBarsService*: to directly retrieve daily data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Daily.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run GAFAM sample

### Step 1: Install packages

In [ ]:
#r "nuget:Systemathics.Apis"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Daily.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
var token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN");

// Add token to the request header
var headers = new Metadata();
headers.Add("Authorization", $"Bearer {token}");

// Define API entry
var channel = GrpcChannel.ForAddress("https://apis.systemathics.cloud");

In [ ]:
// Display authentication token 
display(token);

### Step 3: Create and process request

#### 3.1 Request Parameters
The following code snippet ensure required objects are created and proceed to request instantiation.
Here is the list of required objects:
* Time period selection: select start and end dates
* *DailyBarsService* in order to retrieve daily data

In [ ]:
// Set time period
// If the time interval is not specified, the service will answer with all the full history
var today = DateTime.Today;
var startDate =  new Date { Year = 2010, Month = 01, Day = 01 };
var endDate = new Date { Year = today.Year, Month = today.Month, Day = today.Day };

// Build the daily data request date interval (we are using Google date time format : we have to cast the dates)
var dateIntervals = new DateInterval() { StartDate = startDate, EndDate = endDate};

#### 3.2 Request creation

In [ ]:
// Generate contraints based on previous dates selections
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);

In [ ]:
// Instantiate the daily data service
var service = new DailyBarsService.DailyBarsServiceClient(channel);

#### 3.3 Retrieve components close prices
The following code snippet defines a method that retrieves a *close prices* over time for a given instrument. This method takes *DailyData reply* as per input and returns a *dates* and *close prices* dictionary.

In [ ]:
// Create method to retrieve close prices time series from daily data response
public static Dictionary<DateTime, double> GetPrices(DailyBar[] data)
{
    var pricesDict = new Dictionary<DateTime, double>(); 
    foreach (var reply in data)
    {
        var date = new DateTime(reply.Date.Year, reply.Date.Month, reply.Date.Day);
        pricesDict[date] = reply.Close;
    }
    return pricesDict;
}

The following code snippet defines a method that generates a *daily data request* for a given instrument. This method takes *Exchange* and *Ticker* as per inputs and returns a *DailyRequest*.

In [ ]:
// Create method to generate daily data request for each instrument
public static DailyBarsRequest GenerateRequest(string exchange, string ticker, Constraints constraints)
{
    return new DailyBarsRequest {Identifier = new Identifier { Exchange = exchange , Ticker = ticker}, Constraints = constraints};
}

The following code snippet retrieves *close prices* over time for GAFAM instruments by calling the previous methods and the *dailyService* in order to retrieve daily data.

In [ ]:
var pricesByInstrument = new Dictionary<string, Dictionary<DateTime, double>>();
var tickers = new List<string> {"GOOGL", "AAPL", "FB", "AMZN", "MSFT"};

foreach (var ticker in tickers)
{
    var request = GenerateRequest("XNGS", ticker, constraints);
    var reply = service.DailyBars(request, headers);
    var prices = GetPrices(reply.Data.ToArray());
    pricesByInstrument[ticker] = prices;
}

#### 3.4 Compute GAFAM index
The following code snippets computes the GAFAM index:

In [ ]:
// Merge dates in a single list
// Not all the instruments are quoted at the same time
// For example : Facebook starts trading in 2012, Google before 2010
// We will compute our index based on available assets over the time
var allDates = new List<DateTime>();
foreach (var elt in pricesByInstrument.Keys) 
{
    var tmpdict = pricesByInstrument[elt];
    foreach (var key in tmpdict.Keys) 
    {
        allDates.Add(key);
    }
}
var dates = allDates.Distinct().OrderBy(d => d);
//display(dates.ToArray())

In [ ]:
// Compute the index : equal weight index
var indexValues = new Dictionary<DateTime, double>();

// Weight by instruments over the days
var weightByInstrument = new Dictionary<string, Dictionary<DateTime, double>>();

foreach( var instrument in pricesByInstrument.Keys)
{
    weightByInstrument[instrument] = new Dictionary<DateTime, double>();
}

// Build the index and the weights for all the instruments over all the days
foreach (var date in dates) 
{
    // Index price
    var index = 0.0;
    var counter = 0;
    foreach( var instrument in pricesByInstrument.Keys)
    {
        var test = pricesByInstrument[instrument].TryGetValue(date, out var tmp);
        if (test) 
        {
            index += tmp;
            counter++;
        }
    }

    index =  counter > 0 ? index / counter : 0.0;
    indexValues[date] = index;
    
    // Weighted price for all the instruments
    foreach( var instrument in pricesByInstrument.Keys)
    {
        var test = pricesByInstrument[instrument].TryGetValue(date, out var tmp);
        if(test)
        {
            weightByInstrument[instrument][date] = Math.Log(tmp / index);
        }
    }
}

//display(indexValues);
//display(weightByInstrument);

### Step 4: Visualize data

#### 4.1 Plot GAFAM components Vs index
*`XPlot.Plotly`* is used as per open source display package.

In [ ]:
// Create the graph
// Add the index on the first yaxis
var gafamGraph = new List<Scattergl>
{
    new Scattergl
    {
        name = "Index",
        mode = "lines",
        x = indexValues.Keys,
        y = indexValues.Values,
        yaxis = "y2"
    }
};

// Add the instruments on the second yaxis
foreach( var instrument in pricesByInstrument.Keys)
{
    var g = new Scattergl
    {
        name = instrument,
        mode = "lines",
        x = weightByInstrument[instrument].Keys,
        y = weightByInstrument[instrument].Values
    };
    
    gafamGraph.Add(g);  
}

var layout = new Layout.Layout{ 
    title = "GAFAM",
    yaxis = new Yaxis {
        title = "Weighted prices", 
        autorange = true,
        side = "left",
    },
    yaxis2 = new Yaxis {
        title = "Index", 
        autorange = true,
        side = "right",  
    }
};

var chart = Chart.Plot(gafamGraph, layout);
chart.WithXTitle($"Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);